<a href="https://colab.research.google.com/github/base-datos-empresas/MixedRealityToolkit-Unity/blob/Troy-Ferrell-patch-3/Verificador_de_emails_de_bases_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar dependencias
!pip install dnspython openpyxl

import os
import pandas as pd
import re
import dns.resolver
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from google.colab import files

# Instrucciones de uso en consola
def mostrar_instrucciones():
    instrucciones = """
    Instrucciones de uso:
    1. Suba un archivo Excel que contenga una columna llamada 'email' con las direcciones de correo electrónico a verificar.
    2. Elija el modo de verificación:
       - '1' para modo restringido (solo los primeros 40 correos).
       - '2' para modo completo (todos los correos).
    3. Obtendrá dos archivos de salida:
       - Un archivo con la verificación normal (formato y MX).
       - Un archivo con la verificación avanzada (formato, MX, SPF, DMARC y DKIM).

    Nota: Este código ha sido proporcionado por centraldecomunicacion.es.
    Puede obtener bases de datos de empresas de forma gratuita y premium en: https://www.centraldecomunicacion.es/
    """
    print(instrucciones)

# Funciones de verificación
def verificar_formato_email(email):
    """
    Verifica que el formato del email sea correcto.
    """
    patron_email = re.compile(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')
    return patron_email.match(email) is not None

def verificar_dominio_y_MX(email):
    """
    Verifica que el dominio del email tenga registros DNS y MX válidos.
    """
    try:
        if '@' not in email:
            return False
        dominio = email.split('@')[-1]
        patron_dominio = re.compile(r'^[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')
        if not patron_dominio.match(dominio):
            return False
        if any(len(etiqueta) > 63 for etiqueta in dominio.split('.')):
            return False
        dns.resolver.resolve(dominio, 'A')
        registros_mx = dns.resolver.resolve(dominio, 'MX')
        return len(registros_mx) > 0
    except dns.exception.DNSException as e:
        print(f"\n\n=== WARNING: DNS Exception ===\nProblematic domain: {dominio}\nError: {e}")
        return False

def verificar_existencia_email_normal(email):
    """
    Verifica la existencia del correo electrónico con comprobación normal (formato y MX).
    """
    if verificar_formato_email(email):
        return verificar_dominio_y_MX(email)
    return False

def verificar_registros_SPF(dominio):
    """
    Verifica si el dominio tiene registros SPF válidos.
    """
    try:
        registros_spf = dns.resolver.resolve(dominio, 'TXT')
        for txt_record in registros_spf:
            if 'v=spf1' in str(txt_record):
                return True
        return False
    except dns.exception.DNSException as e:
        print(f"\n\n=== WARNING: SPF Exception ===\nProblematic domain: {dominio}\nError: {e}")
        return False

def verificar_registros_DMARC(dominio):
    """
    Verifica si el dominio tiene una política DMARC.
    """
    try:
        registros_dmarc = dns.resolver.resolve('_dmarc.' + dominio, 'TXT')
        for txt_record in registros_dmarc:
            if 'v=DMARC1' in str(txt_record):
                return True
        return False
    except dns.exception.DNSException as e:
        print(f"\n\n=== WARNING: DMARC Exception ===\nProblematic domain: {dominio}\nError: {e}")
        return False

def verificar_registros_DKIM(dominio):
    """
    Verifica si el dominio tiene registros DKIM.
    """
    try:
        selector = 'default'  # Cambiar esto dependiendo del selector DKIM utilizado
        registros_dkim = dns.resolver.resolve(selector + '._domainkey.' + dominio, 'TXT')
        for txt_record in registros_dkim:
            if 'v=DKIM1' in str(txt_record):
                return True
        return False
    except dns.exception.DNSException as e:
        print(f"\n\n=== WARNING: DKIM Exception ===\nProblematic domain: {dominio}\nError: {e}")
        return False

def verificar_existencia_email_avanzada(email):
    """
    Verifica la existencia del correo electrónico con comprobación avanzada (formato, MX, SPF, DMARC y DKIM).
    """
    if verificar_formato_email(email):
        dominio = email.split('@')[-1]
        if verificar_dominio_y_MX(email):
            spf = verificar_registros_SPF(dominio)
            dmarc = verificar_registros_DMARC(dominio)
            dkim = verificar_registros_DKIM(dominio)
            if spf and dmarc and dkim:
                return True
        else:
            print(f"Fallo en la verificación de dominio/MX para: {email}")
    else:
        print(f"Fallo en la verificación de formato para: {email}")
    return False

# Función para verificar y limpiar emails en un DataFrame
def verificar_y_limpiar_emails(df, avanzada=False):
    """
    Verifica y limpia los correos electrónicos en el DataFrame.
    """
    if 'email' not in df.columns:
        print("La columna 'email' no existe en este DataFrame.")
        return df, 0, 0

    total_limpieza = 0
    total_verificados = 0
    emails_a_verificar = df['email']

    for idx, email in emails_a_verificar.items():
        if not pd.isna(email):
            total_verificados += 1
            if avanzada:
                if not verificar_existencia_email_avanzada(email):
                    df.at[idx, 'email'] = ''
                    total_limpieza += 1
            else:
                if not verificar_existencia_email_normal(email):
                    df.at[idx, 'email'] = ''
                    total_limpieza += 1

    return df, total_limpieza, total_verificados

# Función para aplicar modificaciones en la hoja del archivo Excel
def aplicar_modificaciones_en_hoja(hoja, df_modificado):
    """
    Aplica las modificaciones en la hoja del archivo Excel.
    """
    for row_idx, row in enumerate(dataframe_to_rows(df_modificado, index=False, header=True), start=1):
        for col_idx, value in enumerate(row, start=1):
            hoja.cell(row=row_idx, column=col_idx, value=value)

# Función principal para procesar el archivo subido
def procesar_archivo(archivo, modo_restringido):
    """
    Procesa el archivo subido, verifica y limpia los correos electrónicos.
    """
    nombre_archivo_base, extension_archivo = os.path.splitext(archivo)
    ruta_archivo_final_normal = nombre_archivo_base + '_NORMAL' + extension_archivo
    ruta_archivo_final_avanzada = nombre_archivo_base + '_AVANZADA' + extension_archivo

    with pd.ExcelFile(archivo, engine='openpyxl') as xls:
        writer_normal = pd.ExcelWriter(ruta_archivo_final_normal, engine='openpyxl')
        writer_avanzada = pd.ExcelWriter(ruta_archivo_final_avanzada, engine='openpyxl')

        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet_name)
            if sheet_name == xls.sheet_names[0]:  # Solo modificar la primera hoja
                df_modificado_normal, limpieza_normal, verificados_normal = verificar_y_limpiar_emails(df, avanzada=False)
                df_modificado_avanzada, limpieza_avanzada, verificados_avanzados = verificar_y_limpiar_emails(df, avanzada=True)
                df_modificado_normal.to_excel(writer_normal, sheet_name=sheet_name, index=False)
                df_modificado_avanzada.to_excel(writer_avanzada, sheet_name=sheet_name, index=False)
            else:
                df.to_excel(writer_normal, sheet_name=sheet_name, index=False)
                df.to_excel(writer_avanzada, sheet_name=sheet_name, index=False)

        writer_normal.close()
        writer_avanzada.close()

        # Aplicar auto filtro y guardar
        if os.path.exists(ruta_archivo_final_normal):
            libro_normal = openpyxl.load_workbook(ruta_archivo_final_normal)
            hoja_normal = libro_normal[xls.sheet_names[0]]
            hoja_normal.auto_filter.ref = hoja_normal.dimensions
            aplicar_modificaciones_en_hoja(hoja_normal, df_modificado_normal)
            libro_normal.save(ruta_archivo_final_normal)

        if os.path.exists(ruta_archivo_final_avanzada):
            libro_avanzada = openpyxl.load_workbook(ruta_archivo_final_avanzada)
            hoja_avanzada = libro_avanzada[xls.sheet_names[0]]
            hoja_avanzada.auto_filter.ref = hoja_avanzada.dimensions
            aplicar_modificaciones_en_hoja(hoja_avanzada, df_modificado_avanzada)
            libro_avanzada.save(ruta_archivo_final_avanzada)

    # Descargar archivos
    files.download(ruta_archivo_final_normal)
    files.download(ruta_archivo_final_avanzada)

    return limpieza_normal, verificados_normal, limpieza_avanzada, verificados_avanzados

# Mostrar instrucciones
mostrar_instrucciones()

# Cargar archivo
uploaded = files.upload()
for filename in uploaded.keys():
    print(f'Archivo subido: {filename}')
    modo_restringido = input("Elige el modo (escribe '1' para modo restringido o '2' para modo completo): ").strip() == '1'
    limpieza_normal, verificados_normal, limpieza_avanzada, verificados_avanzados = procesar_archivo(filename, modo_restringido)
    print(f"Total de correos electrónicos comprobados (normal): {verificados_normal}")
    print(f"Total de correos electrónicos auténticos (normal): {verificados_normal - limpieza_normal}")
    print(f"Total de correos electrónicos comprobados (avanzada): {verificados_avanzados}")
    print(f"Total de correos electrónicos auténticos (avanzada): {verificados_avanzados - limpieza_avanzada}")

print("\nEste código ha sido proporcionado por centraldecomunicacion.es.")
print("Puede obtener bases de datos de empresas de forma gratuita y premium en: https://www.centraldecomunicacion.es/")



    Instrucciones de uso:
    1. Suba un archivo Excel que contenga una columna llamada 'email' con las direcciones de correo electrónico a verificar.
    2. Elija el modo de verificación: 
       - '1' para modo restringido (solo los primeros 40 correos).
       - '2' para modo completo (todos los correos).
    3. Obtendrá dos archivos de salida:
       - Un archivo con la verificación normal (formato y MX).
       - Un archivo con la verificación avanzada (formato, MX, SPF, DMARC y DKIM).
    
    Nota: Este código ha sido proporcionado por centraldecomunicacion.es.
    Puede obtener bases de datos de empresas de forma gratuita y premium en: https://www.centraldecomunicacion.es/
    


Saving Interiorismo CENTRAL vcen_CALIDADVERIFICADA.xlsx to Interiorismo CENTRAL vcen_CALIDADVERIFICADA (1).xlsx
Archivo subido: Interiorismo CENTRAL vcen_CALIDADVERIFICADA (1).xlsx
Elige el modo (escribe '1' para modo restringido o '2' para modo completo): 2


=== WARNING: DNS Exception ===
Problematic domain: axtresestudio.es
Error: The DNS query name does not exist: axtresestudio.es.


=== WARNING: DKIM Exception ===
Problematic domain: gmail.com
Error: The DNS query name does not exist: default._domainkey.gmail.com.


=== WARNING: DMARC Exception ===
Problematic domain: laam.es
Error: The DNS query name does not exist: _dmarc.laam.es.


=== WARNING: DKIM Exception ===
Problematic domain: laam.es
Error: The DNS query name does not exist: default._domainkey.laam.es.


=== WARNING: DMARC Exception ===
Problematic domain: laam.es
Error: The DNS query name does not exist: _dmarc.laam.es.


=== WARNING: DKIM Exception ===
Problematic domain: laam.es
Error: The DNS query name does not exis

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Total de correos electrónicos comprobados (normal): 298
Total de correos electrónicos auténticos (normal): 297
Total de correos electrónicos comprobados (avanzada): 298
Total de correos electrónicos auténticos (avanzada): 43

Este código ha sido proporcionado por centraldecomunicacion.es.
Puede obtener bases de datos de empresas de forma gratuita y premium en: https://www.centraldecomunicacion.es/
